In [2]:
# the [5] ins local mens that it will be a partition in our spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('PartData') \
    .master('local[5]').getOrCreate()

In [3]:
# We create a simple range and check the number of partitions.
df = spark.range(0,20)
df.rdd.getNumPartitions()
# by default we get 5 partitions.

5

In [4]:
# We can specify the partition number directly using  parameters.
# In this case we set 10 partition for rdd1.
rdd1 = spark.sparkContext.parallelize((0,20),10)
rdd1.getNumPartitions()
# No we get 10 partitions

10

In [7]:
rddFromFile = spark.sparkContext \
    .textFile('/home/lastorder/Documents/curso-apache-spark-platzi/files/deporte.csv', 10)


In [8]:
rddFromFile.getNumPartitions()

10

In [10]:
# To save our RDDs and Dataframes in an specific folder
rddFromFile.saveAsTextFile('/home/lastorder/Documents/curso-apache-spark-platzi/output_files/')

In [11]:
!ls /home/lastorder/Documents/curso-apache-spark-platzi/output_files/

part-00000  part-00002	part-00004  part-00006	part-00008  _SUCCESS
part-00001  part-00003	part-00005  part-00007	part-00009


In [12]:
!head -n 10 /home/lastorder/Documents/curso-apache-spark-platzi/output_files/part-00000

deporte_id,deporte
1,Basketball
2,Judo
3,Football
4,Tug-Of-War
5,Speed Skating
6,Cross Country Skiing


In [13]:
# To get the values one option is to use:
rdd = spark.sparkContext.wholeTextFiles('/home/lastorder/Documents/curso-apache-spark-platzi/output_files/*')

In [14]:
# We can see that this is no the best way to get partitional storage data.
rdd.take(4)

[('file:/home/lastorder/Documents/curso-apache-spark-platzi/output_files/part-00000',
  'deporte_id,deporte\n1,Basketball\n2,Judo\n3,Football\n4,Tug-Of-War\n5,Speed Skating\n6,Cross Country Skiing\n'),
 ('file:/home/lastorder/Documents/curso-apache-spark-platzi/output_files/part-00001',
  '7,Athletics\n8,Ice Hockey\n9,Swimming\n10,Badminton\n11,Sailing\n12,Biathlon\n13,Gymnastics\n14,Art Competitions\n'),
 ('file:/home/lastorder/Documents/curso-apache-spark-platzi/output_files/part-00002',
  '15,Alpine Skiing\n16,Handball\n17,Weightlifting\n18,Wrestling\n19,Luge\n20,Water Polo\n'),
 ('file:/home/lastorder/Documents/curso-apache-spark-platzi/output_files/part-00003',
  '21,Hockey\n22,Rowing\n23,Bobsleigh\n24,Fencing\n25,Equestrianism\n26,Shooting\n27,Boxing\n28,Taekwondo\n')]

In [16]:
# This is en exeption where we can use collect in Spark
rdd_list = rdd.mapValues(lambda x: x.split()).collect()

In [18]:
# we neet to extrect only the path to the files from the list
rdd_list = [l[0] for l in rdd_list]
rdd_list.sort()

In [19]:
rdd_list

['file:/home/lastorder/Documents/curso-apache-spark-platzi/output_files/part-00000',
 'file:/home/lastorder/Documents/curso-apache-spark-platzi/output_files/part-00001',
 'file:/home/lastorder/Documents/curso-apache-spark-platzi/output_files/part-00002',
 'file:/home/lastorder/Documents/curso-apache-spark-platzi/output_files/part-00003',
 'file:/home/lastorder/Documents/curso-apache-spark-platzi/output_files/part-00004',
 'file:/home/lastorder/Documents/curso-apache-spark-platzi/output_files/part-00005',
 'file:/home/lastorder/Documents/curso-apache-spark-platzi/output_files/part-00006',
 'file:/home/lastorder/Documents/curso-apache-spark-platzi/output_files/part-00007',
 'file:/home/lastorder/Documents/curso-apache-spark-platzi/output_files/part-00008',
 'file:/home/lastorder/Documents/curso-apache-spark-platzi/output_files/part-00009']

In [20]:
#we are indicate that join all files in the path with a comma
rdd_fromList = spark.sparkContext.textFile(','.join(rdd_list), 10) \
    .map(lambda l: l.split(','))

In [21]:
# Checking the result
rdd_fromList.take(4)

[['deporte_id', 'deporte'],
 ['1', 'Basketball'],
 ['2', 'Judo'],
 ['3', 'Football']]